In [8]:
import os
import pandas as pd
from tqdm import tqdm
from ase import io
from ase.md.langevin import Langevin
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase import units
from chgnet.model.dynamics import CHGNetCalculator

# === ✅ STEP 0: Set working directory to where your folders are ===
os.chdir("/home/mehuldarak/summer")  # ← change this to your actual path!

# === STEP 1: Configuration ===
relaxed_root = "relax_final"
md_root = "md"
traj_dir = os.path.join(md_root, "mdtraj")
cif_dir = os.path.join(md_root, "mdcifs")
os.makedirs(traj_dir, exist_ok=True)
os.makedirs(cif_dir, exist_ok=True)

temperatures = [300, 450]
timestep = 1 * units.fs
n_steps = 1000
snapshot_interval = 5
friction = 0.01

# === STEP 2: Metadata parser ===
def parse_filename_metadata(fname):
    try:
        tokens = os.path.splitext(fname)[0].split("_")
        return dict(
            full_name=fname,
            llzo_dir=tokens[2],
            llzo_termination=tokens[3],
            llzo_order=tokens[4],
            stoichiometry=tokens[5],
            li_dir=tokens[7],
            notes=""
        )
    except Exception as e:
        return dict(full_name=fname, notes=f"parse_error: {str(e)}")

# === STEP 3: MD runner ===
def run_md_task(cif_path, base_name, temperature):
    print(f"🔥 Running MD: {base_name} @ {temperature}K")
    atoms = io.read(cif_path)
    atoms.calc = CHGNetCalculator()
    MaxwellBoltzmannDistribution(atoms, temperature_K=temperature)

    traj_path = os.path.join(traj_dir, f"{base_name}_T{temperature}.traj")
    traj_writer = io.Trajectory(traj_path, 'w', atoms)

    snapshots = []

    def save_snapshot(a=atoms):
        traj_writer.write(a)
        snapshots.append(a.copy())
        print(f"[snapshot] {len(snapshots)}", flush=True)

    dyn = Langevin(atoms, timestep, temperature_K=temperature, friction=friction)
    dyn.attach(save_snapshot, interval=snapshot_interval)
    dyn.run(n_steps)
    traj_writer.close()

    print(f"✅ Done: {base_name}_T{temperature}.traj | Snapshots: {len(snapshots)}")

    for i, snap in enumerate(snapshots):
        cif_file = f"{base_name}_T{temperature}_{i:04d}.cif"
        cif_path_out = os.path.join(cif_dir, cif_file)
        io.write(cif_path_out, snap)
        if i in [0, len(snapshots) - 1]:
            print(f"[CIF] {cif_file}")

    return dict(
        base_name=base_name,
        temperature_K=temperature,
        n_snapshots=len(snapshots),
        source_path=cif_path
    )

# === STEP 4: Main Loop
results = []
cif_jobs = []

for root, _, files in os.walk(relaxed_root):
    for fname in files:
        if fname.endswith(".cif") and fname.startswith("cellrelaxed_"):
            cif_path = os.path.join(root, fname)
            base_name = os.path.splitext(fname)[0]
            parsed = parse_filename_metadata(fname)
            for T in temperatures:
                cif_jobs.append((cif_path, base_name, T, parsed))

print(f"📦 Total jobs: {len(cif_jobs)}")

for cif_path, base_name, T, parsed in tqdm(cif_jobs):
    result = run_md_task(cif_path, base_name, T)
    result.update(parsed)
    results.append(result)

# === STEP 5: Save Excel
df = pd.DataFrame(results)
excel_path = os.path.join(md_root, "mdinfo.xlsx")
df.to_excel(excel_path, index=False)

print("\n✅ All MD runs done.")
print(f"📁 Trajs → {traj_dir}")
print(f"📁 Cifs  → {cif_dir}")
print(f"📝 Info  → {excel_path}")


📦 Total jobs: 30


  0%|          | 0/30 [00:00<?, ?it/s]

🔥 Running MD: cellrelaxed_LLZO_010_Li_order4_off__Li_110_slab_heavy @ 300K
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
[snapshot] 1
[snapshot] 2
[snapshot] 3
[snapshot] 4
[snapshot] 5
[snapshot] 6
[snapshot] 7
[snapshot] 8
[snapshot] 9
[snapshot] 10
[snapshot] 11
[snapshot] 12
[snapshot] 13
[snapshot] 14
[snapshot] 15
[snapshot] 16
[snapshot] 17
[snapshot] 18
[snapshot] 19
[snapshot] 20
[snapshot] 21
[snapshot] 22
[snapshot] 23
[snapshot] 24
[snapshot] 25
[snapshot] 26
[snapshot] 27
[snapshot] 28
[snapshot] 29
[snapshot] 30
[snapshot] 31
[snapshot] 32
[snapshot] 33
[snapshot] 34
[snapshot] 35
[snapshot] 36
[snapshot] 37
[snapshot] 38
[snapshot] 39
[snapshot] 40
[snapshot] 41
[snapshot] 42
[snapshot] 43
[snapshot] 44
[snapshot] 45
[snapshot] 46
[snapshot] 47
[snapshot] 48
[snapshot] 49
[snapshot] 50
[snapshot] 51
[snapshot] 52
[snapshot] 53
[snapshot] 54
[snapshot] 55
[snapshot] 56
[snapshot] 57
[snapshot] 58
[snapshot] 59
[snapshot] 60
[snapshot] 61
[snaps

  3%|▎         | 1/30 [04:09<2:00:41, 249.69s/it]

[CIF] cellrelaxed_LLZO_010_Li_order4_off__Li_110_slab_heavy_T300_0200.cif
🔥 Running MD: cellrelaxed_LLZO_010_Li_order4_off__Li_110_slab_heavy @ 450K
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
[snapshot] 1
[snapshot] 2
[snapshot] 3
[snapshot] 4
[snapshot] 5
[snapshot] 6
[snapshot] 7
[snapshot] 8
[snapshot] 9
[snapshot] 10
[snapshot] 11
[snapshot] 12
[snapshot] 13
[snapshot] 14
[snapshot] 15
[snapshot] 16
[snapshot] 17
[snapshot] 18
[snapshot] 19
[snapshot] 20
[snapshot] 21
[snapshot] 22
[snapshot] 23
[snapshot] 24
[snapshot] 25
[snapshot] 26
[snapshot] 27
[snapshot] 28
[snapshot] 29
[snapshot] 30
[snapshot] 31
[snapshot] 32
[snapshot] 33
[snapshot] 34
[snapshot] 35
[snapshot] 36
[snapshot] 37
[snapshot] 38
[snapshot] 39
[snapshot] 40
[snapshot] 41
[snapshot] 42
[snapshot] 43
[snapshot] 44
[snapshot] 45
[snapshot] 46
[snapshot] 47
[snapshot] 48
[snapshot] 49
[snapshot] 50
[snapshot] 51
[snapshot] 52
[snapshot] 53
[snapshot] 54
[snapshot] 55
[snapshot] 56
[s

...

[CIF] cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T450_0200.cif
🔥 Running MD: cellrelaxed_LLZO_001_Zr_code93_sto__Li_110_slab_heavy @ 300K
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
[snapshot] 1
[snapshot] 2
[snapshot] 3
[snapshot] 4
[snapshot] 5
[snapshot] 6
[snapshot] 7
[snapshot] 8
[snapshot] 9
[snapshot] 10
[snapshot] 11
[snapshot] 12
[snapshot] 13
[snapshot] 14
[snapshot] 15
[snapshot] 16
[snapshot] 17
[snapshot] 18
[snapshot] 19
[snapshot] 20
[snapshot] 21
[snapshot] 22
[snapshot] 23
[snapshot] 24
[snapshot] 25
[snapshot] 26
[snapshot] 27
[snapshot] 28
[snapshot] 29
[snapshot] 30
[snapshot] 31
[snapshot] 32
[snapshot] 33
[snapshot] 34
[snapshot] 35
[snapshot] 36
[snapshot] 37
[snapshot] 38
[snapshot] 39
[snapshot] 40
[snapshot] 41
[snapshot] 42
[snapshot] 43
[snapshot] 44
[snapshot] 45
[snapshot] 46
[snapshot] 47
[snapshot] 48
[snapshot] 49
[snapshot] 50
[snapshot] 51
[snapshot] 52
[snapshot] 53
[snapshot] 54
[snapshot] 55
[snapshot] 56
[s

 97%|█████████▋| 29/30 [2:48:45<05:49, 349.02s/it]

[CIF] cellrelaxed_LLZO_001_Zr_code93_sto__Li_110_slab_heavy_T300_0200.cif
🔥 Running MD: cellrelaxed_LLZO_001_Zr_code93_sto__Li_110_slab_heavy @ 450K
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
[snapshot] 1
[snapshot] 2
[snapshot] 3
[snapshot] 4
[snapshot] 5
[snapshot] 6
[snapshot] 7
[snapshot] 8
[snapshot] 9
[snapshot] 10
[snapshot] 11
[snapshot] 12
[snapshot] 13
[snapshot] 14
[snapshot] 15
[snapshot] 16
[snapshot] 17
[snapshot] 18
[snapshot] 19
[snapshot] 20
[snapshot] 21
[snapshot] 22
[snapshot] 23
[snapshot] 24
[snapshot] 25
[snapshot] 26
[snapshot] 27
[snapshot] 28
[snapshot] 29
[snapshot] 30
[snapshot] 31
[snapshot] 32
[snapshot] 33
[snapshot] 34
[snapshot] 35
[snapshot] 36
[snapshot] 37
[snapshot] 38
[snapshot] 39
[snapshot] 40
[snapshot] 41
[snapshot] 42
[snapshot] 43
[snapshot] 44
[snapshot] 45
[snapshot] 46
[snapshot] 47
[snapshot] 48
[snapshot] 49
[snapshot] 50
[snapshot] 51
[snapshot] 52
[snapshot] 53
[snapshot] 54
[snapshot] 55
[snapshot] 56
[s

100%|██████████| 30/30 [2:57:49<00:00, 355.65s/it]

[CIF] cellrelaxed_LLZO_001_Zr_code93_sto__Li_110_slab_heavy_T450_0200.cif

✅ All MD runs done.
📁 Trajs → md/mdtraj
📁 Cifs  → md/mdcifs
📝 Info  → md/mdinfo.xlsx
